# DSPy-Clarifai lm and retriever example notebook

This notebook will walk you through on the integration of clarifai into DSPy which enables the DSPy users to leverage clarifai capabilities of calling llm models from clarifai platform and to utilize clarifai app as retriever for their vector search use cases.

### Setup

In [ ]:
!pip install clarifai

In [ ]:
!pip install dspy-ai

Import necessary packages

In [4]:
import dspy
from dspy.retrieve.clarifai_rm import ClarifaiRM 

#### Initialize clarifai app id, user id and PAT.
Create an AI app in clarifai portal in < 1 min by following this link [getting started](https://docs.clarifai.com/clarifai-basics/quick-start/your-first-predictions).

You can browse the portal to obtain [MODEL URL](https://clarifai.com/explore/models) for different models in clarifai community.

In [6]:
#for the demo we are going with llama2-70b-chat
MODEL_URL = "https://clarifai.com/meta/Llama-2/models/llama2-70b-chat" 
PAT = "CLARIFAI_PAT"
USER_ID = "YOUR_ID"
APP_ID = "YOUR_APP"

### Data ingestion into clarifai vectordatabase

To use clarifai as retriever all you have to do is ingest the documents into clarifai app that serves as your vectordatabase to retrieve similar documents.
To simplify the ingestion, we are utilising the clarifaivectordatabase integration for ingestion.

In [ ]:
#run this block to ingest the documents into clarifai app as chunks.
# if you encounter any issue, make sure to run `pip install langchain`

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Clarifai as clarifaivectorstore

loader = TextLoader("YOUR_TEXT_FILE") #replace with your file path
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

clarifai_vector_db = clarifaivectorstore.from_documents(
    user_id=USER_ID,
    app_id=APP_ID,
    documents=docs,
    pat=PAT
)

#### Initialize LLM class

Make sure to pass all the model parameters in inference_params field of clarifaiLLM class. 

In [55]:

llm=dspy.Clarifai(model=MODEL_URL, api_key=PAT, n=2, inference_params={"max_tokens":100,'temperature':0.6})

Initialize Clarifai Retriever model class


In [56]:

retriever_model=ClarifaiRM(clarifai_user_id=USER_ID, clarfiai_app_id=APP_ID, clarifai_pat=PAT, k=2)

configure dspy with llm and rm models.

In [57]:
dspy.settings.configure(lm=llm, rm=retriever_model)

### Example: dspy.signature and dspy.module with clairfaiLLM

In [11]:
sentence = "disney again ransacks its archives for a quick-buck sequel ."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment')
print(classify(sentence=sentence).sentiment)

NEGATIVE




### Example: Quick glimpse into how our retriever works when a query is passed to the dspy.Retrieve class

Here we have used the rulebook of Formula student Germany competition.

link : https://www.formulastudent.de/fileadmin/user_upload/all/2024/rules/FS-Rules_2024_v1.0.pdf 

We have used the .txt version of the file for our demo.

In [51]:
retrieve = dspy.Retrieve()
topK_passages = retrieve("can I test my vehicle engine in pit?").passages

In [52]:
print(topK_passages)

['A 6.8.3\n\nCranking engines in the pits is allowed, when the following conditions are met:\n• The vehicle has passed mechanical inspection.\n• The driven axles are securely jacked up.\n• Gearbox is in neutral.\n• All driven wheels are removed.\n• Connectors to all injectors and ignition coils are detached.\n• A fire extinguisher must be placed next to the engine.\n\nA 6.9\n\nFueling and Oil\n\nA 6.9.1\n\nFueling may only take place at the fuel station and must be conducted by officials only.\n\nA 6.9.2\n\nOpen fuel containers are not permitted at the competition.\n\nA 6.9.3\n\nWaste oil must be taken to the fuel station for disposal.\n\nA 6.10\n\n[EV ONLY ] Working on the Vehicle\n\nA 6.10.1\n\nAll activities require the TSAL to be green.\n\nA 6.10.2\n\nA prominent manual sign indicating the “TSAL green” state must be present whenever the\nLVS is switched off and the requirements for an only green TSAL according to EV 4.10 are\nmet.\n\nA 6.10.3', 'A 6.8.3\n\nCranking engines in the p

## RAG dspy module using clarifai as retriever

Generally to construct a module in dspy, you might need to define 

Signature: 
explain the input and output fields in an intuitive way with just few words.
("question"-> "answer")

Module:
Module can be something where you put the signatures into action by defining a certain module which compiles and generate response for you for the given query.

Construct a signaturre class, which defines the input fields and output fields needed. 
Also, give docstrings and description in verbose, so that the dspy signature could understand the context and compile best prompt for the usecase.

In [53]:
class GenerateAnswer(dspy.Signature):
    """Think and Answer questions based on the context provided."""

    context = dspy.InputField(desc="may contain relevant facts about user query")
    question = dspy.InputField(desc="User query")
    answer = dspy.OutputField(desc="Answer in one or two lines")

Define the module with the actions needs to be performed, here we are showing a small RAG use case where we are retrieving similar contexts using our retriever class and generating response based on the factual context using one of the DSPy module `ChainOfThought`.

In [54]:
class RAG(dspy.Module):
    def __init__(self):
        super().__init__()

        self.retrieve = dspy.Retrieve()
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

Now we are passing our query and retrieving relevant chunks using clarifai retriever and based on factual evidence, model is able to generate response.

In [59]:
# Ask any question you like to this RAG program.
my_question = "can I test my vehicle engine in pit before inspection?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
Rag_obj= RAG()
predict_response_llama70b=Rag_obj(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {predict_response_llama70b.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in predict_response_llama70b.context]}")

Question: can I test my vehicle engine in pit before inspection?
Predicted Answer: No, you cannot test your vehicle engine in the pit before inspection.
Retrieved Contexts (truncated): ['A 6.8.3\n\nCranking engines in the pits is allowed, when the following conditions are met:\n• The vehicle has passed mechanical inspection.\n• The driven axles are securely jacked up.\n• Gearbox is in neut...', 'A 6.8.3\n\nCranking engines in the pits is allowed, when the following conditions are met:\n• The vehicle has passed mechanical inspection.\n• The driven axles are securely jacked up.\n• Gearbox is in neut...']


### Now we will compare our RAG DSPy module with different community models from clarifai and comapare responses

### Mistral-7b Instruct

In [64]:
mistral_lm = dspy.Clarifai(model="https://clarifai.com/mistralai/completion/models/mistral-7B-Instruct", api_key=PAT, n=2, inference_params={'temperature':0.6})
dspy.settings.configure(lm=mistral_lm, rm=retriever_model)

In [70]:
my_question = "can I test my vehicle engine in pit before inspection?"
Rag_obj= RAG()
predict_response_mistral=Rag_obj(my_question)

print(f"Question: {my_question}")
print(f"Predicted Answer: {predict_response_mistral.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in predict_response_mistral.context]}")

Question: can I test my vehicle engine in pit before inspection?
Predicted Answer: Reasoning: According to the context, cranking engines in the pits is allowed only when the vehicle has passed mechanical inspection.

Answer: No, you cannot test your vehicle engine in pit before inspection.
Retrieved Contexts (truncated): ['A 6.8.3\n\nCranking engines in the pits is allowed, when the following conditions are met:\n• The vehicle has passed mechanical inspection.\n• The driven axles are securely jacked up.\n• Gearbox is in neut...', 'A 6.8.3\n\nCranking engines in the pits is allowed, when the following conditions are met:\n• The vehicle has passed mechanical inspection.\n• The driven axles are securely jacked up.\n• Gearbox is in neut...']


### Gemini Pro

In [66]:

gemini_lm = dspy.Clarifai(model="https://clarifai.com/gcp/generate/models/gemini-pro", api_key=PAT, n=2)
dspy.settings.configure(lm=gemini_lm, rm=retriever_model)

In [67]:
my_question = "can I test my vehicle engine in pit before inspection?"
Rag_obj= RAG()
predict_response_gemini=Rag_obj(my_question)

print(f"Question: {my_question}")
print(f"Predicted Answer: {predict_response_gemini.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in predict_response_gemini.context]}")

Question: can I test my vehicle engine in pit before inspection?
Predicted Answer: No, you can't test your vehicle engine in the pits before inspection.
Retrieved Contexts (truncated): ['A 6.8.3\n\nCranking engines in the pits is allowed, when the following conditions are met:\n• The vehicle has passed mechanical inspection.\n• The driven axles are securely jacked up.\n• Gearbox is in neut...', 'A 6.8.3\n\nCranking engines in the pits is allowed, when the following conditions are met:\n• The vehicle has passed mechanical inspection.\n• The driven axles are securely jacked up.\n• Gearbox is in neut...']


Clarifai enables you to test your dspy module with different llm models and compare the response as it is crucial part of prompt engineering to test and achieve the right combination of llm models with right prompt.